# <font class='ign_color'>ROS IN A SINGLE WEEK</font>

# 单元 4: ROS Actions

<table style="width:100%">
  <tr>
    <th>预计完成时间: 2'5 小时<br><br>你将在本单元学到什么？
<ul>
  <li>如何创建一个action server（操作服务器）</li>
  <li>如何创建你的action message</li>
</ul></th>
    <th><img src="img/drone.jpg" width="291" height="248" /></th> 
  </tr>
</table>

## 第二部分

在上一课中，你已经学到如何<b>调用</b>action server。在本课中，你将学习如何<b>创建</b>你自己的action server。

<figure>
  <img id="图-4.5" src="img/action_interface.png"/>
   <center> <figcaption>图.4.5 - Action 接口图 Copy 2</figcaption></center>
</figure>

## 编写action server

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;">练习4.11: 通过Notebook测试 Fibonacci Action Server </p><br>
执行以下Python code代码通过点击IPython notebook右上角的运行按钮。<br> 
<img src="img/font-awesome_step-forward.png" style="float:left"/><br>
<br><br>
你也可以按 <i>[CTRL]+[Enter]</i>来执行它。<br>
<br>

当程序完成后，不要忘记 <span style="color:red">重启Kernel(内核)</span>。 这将清除由ROS通过python程序生成的所有节点。这是必要的，因为python程序只能启动<b>一个</b>rospy节点。因为这个notebook只是一个分割的python脚本，如果你尝试执行两个snipets而不重启内核将会出现rospy异常。 
你可以通过点击图标来做到这一点。<br>
<img src="img/refresh_icon.png" style="float:left"/>
<br><br><br>


以下是一个ROS action server的示例代码。当被调用时，action server将根据给定的顺序生成一个Fibonacci sequence（斐波那契序列）。Action server目标信息必须指明要计算的序列的顺序，反馈是被计算的那个序列，并且结果是最终的斐波那契序列。
<br>
</th>
</tr>
</table>

<p style="background:#3B8F10;color:white;" id="prg-4.11a">Python Program {4.11a}: fibonacci_action_server.py </p><br>

In [1]:
import rospy

import actionlib

from actionlib_tutorials.msg import FibonacciFeedback, FibonacciResult, FibonacciAction

class FibonacciClass(object):
    
  # 创建用于发布feedback/result的消息
  _feedback = FibonacciFeedback()
  _result   = FibonacciResult()

  def __init__(self):
    # 创建 action server
    self._as = actionlib.SimpleActionServer("fibonacci_as", FibonacciAction, self.goal_callback, False)
    self._as.start()
    
  def goal_callback(self, goal):
    # 该回调被调用当 action server 也被调用时
    # 这是计算斐波那契序列的函数
    # 并将恢复序列到调用action server的节点
    
    # 辅助变量
    r = rospy.Rate(1)
    success = True
    
    # 附加给斐波那契序列的根据
    self._feedback.sequence = []
    self._feedback.sequence.append(0)
    self._feedback.sequence.append(1)
    
    # 发布消息到用户的控制台
    rospy.loginfo('"fibonacci_as": Executing, creating fibonacci sequence of order %i with seeds %i, %i' % ( goal.order, self._feedback.sequence[0], self._feedback.sequence[1]))
    
    # 开始计算斐波那契序列
    fibonacciOrder = goal.order
    for i in xrange(1, fibonacciOrder):
    
      # 检查action client请求没有抢占(取消) 
      if self._as.is_preempt_requested():
        rospy.loginfo('The goal has been cancelled/preempted')
        # 下一行，将客户端设置为抢占状态 (目标被取消)
        self._as.set_preempted()
        success = False
        # 结束斐波那契数列的计算
        break
      
      # 构建要下一个要发生的反馈msg
      self._feedback.sequence.append(self._feedback.sequence[i] + self._feedback.sequence[i-1])
      # 发布反馈
      self._as.publish_feedback(self._feedback)
      # 序列以1Hz频率计算
      r.sleep()
    
    # 在这一点上，要么目标已经实现（success == true）
    # 要么客户端抢占目标（success == false）
    # 如果成功，那么我们发布最终结果
    # 如果不成功，我们不会在结果中发布任何内容
    if success:
      self._result.sequence = self._feedback.sequence
      rospy.loginfo('Succeeded calculating the Fibonacci of order %i' % fibonacciOrder )
      self._as.set_succeeded(self._result)
      
if __name__ == '__main__':
  rospy.init_node('fibonacci')
  FibonacciClass()
  rospy.spin()

<p style="background:#3B8F10;color:white;">Python程序代码说明: {4.11a}</p><br>
在这种情况下，action server正在使用被定义名为<i>Fibonacci.action</i>的message。该message已被ROS创建到其 <i>actionlib_tutorials</i>包中。
<p style="background:#3B8F10;color:white;">结束Python程序代码说明: {4.11a}</p><br>

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;">练习 4.12a: 检查Fibonacci action msg结构</p><br>
通过访问<i>actionlib_tutorials</i>包的<i>action</i> 目录，来检查Fibonacci.action消息定义的结构。
</th>
</tr>
</table>

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;">练习 4.12b: 查看从Fibonacci输出的反馈和topic messages结果</p><br>

再次启动上面的python代码 <a href="#prg-4.11a">{4.11a}</a> 以使fibonacci服务器运行。<br>
然后在其相应的WebShells执行以下命令。<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #FFFFFF">在 WebShell #1中执行: Echo Result</p><br>
rostopic echo /fibonacci_as/result<br>
</th>
</tr>
</table>
<br><br><br><br><br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #FFFFFF">在 WebShell #2中执行: Echo the feedback</p><br>
rostopic echo /fibonacci_as/feedback<br>
</th>
</tr>
</table>
<br><br><br><br><br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #FFFFFF">在 WebShell #3中执行: 发生目标到你的fibonacci服务器</p><br>
rostopic pub /fibonacci_as/goal actionlib_tutorials/FibonacciActionGoal [TAB][TAB]<br>
</th>
</tr>
</table>
<br><br><br><br><br>

<p style="background:#3B8F10;color:white;">练习4.12b的数据</p><br>
<ul>
<li>
在Python代码中的messages称为FibonacciGoal， FibonacciResult 和 FibonacciFeedback，而在topics中，它们被称为FibonacciActionGoal， FibonacciActionResult 和 FibonacciActionFeedback。
</li>
<li>
<b>FibonacciGoal</b> 和 <b>FibonacciActionGoal</b>之间有什么区别?
</li>
<li>
区别主要是<b>FibonacciGoal</b>是不完整的，没有一些元素需要直接用于topics，只是用于python对象。而 <b>FibonacciActionGoal</b> 由messages使用的，并且当你直接处理topics时必须要用的。
</li>
</ul>




<p style="background:#AE0202;color:white;">练习 4.12b的预期结果</p><br>
在调用action后，feedback topic应该发布反馈和一旦结束计算的结果。


<br>
</th>
</tr>
</table>

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;" id="ex-4-13">练习 4.13: 创建带有在正方形中移动飞行器的Action Server的包</p><br>
创建包含一个action server的包，可使无人机在被调用时在方形中移动。<br>
通过topics创建action server，并观察结果和反馈。 <br>
基于你之前fibonacci例子中的代码 <a href="#prg-4.11a">{4.11a}</a> ，以及你在练习4.6中做的可在拍摄时移动飞行器的client。<br>

<p style="background:#3B8F10;color:white;">练习 4.13的数据</p>
<ul>
<li>正方形的各边的尺寸应作为整数被指定在目标message。</li>
<li>反馈应该发布在当机器人正在正方形运行的当前秒。</li>
<li>结果应该发布机器人在正方形中运行所使用的总秒数</li>
<li>使用<span style="color: orange"><i>Test.action</i></span> message于该action server。使用shell命令 <span style="color: orange"><i>locate Test.action</i></span> 以找到它定义该message的位置。</li>
</ul>

<p style="background:#AE0202;color:white;">练习4.13的预期结果</p><br>
结果显示在动画中 <a href="#fig-4.6">{图:4.6}</a>

</th>
</tr>
</table>




<figure>
  <img id="fig-4.6" src="img/drone.gif">
   <center> <figcaption>图.4.6 - Ardrone moved through commands issed by an custom action server Ex 4.13</figcaption></center>
</figure>

## 如何创建你自己的action server message

<b>始终建议你使用已由ROS提供的action messages。</b><br>
它们可以在以下ROS包中被找到:
<ul>
<li>actionlib</li>
    <ul>
    <li>Test.action</li>
    <li>TestRequest.action </li>
    <li>TwoInts.action</li>
    </ul>
<li>actionlib_tutorials</li>
    <ul>
    <li>Fibonacci.action</li>
    <li>Averaging.action</li>
    </ul>
</ul>
<br>

尽管如此，如果你需创建你自己的类型，请做以下练习来了解如何创建。

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;">练习 4.14: 创建你自己的自定义action message</p><br>
<ol>
  <li>在包中创建一个 <i>action</i> 目录</li>
  <li>创建你的 NAME.action message 文件
      <ul>
        <li>Action message文件的名字将稍后被确认为类型的名字，以用于 <i>action server</i> 和/或 <i>action client</i></li>
        <li>
        记得 NAME.action 文件必须包含三部分，每一部分由三个连字符分隔。
        #<span style="color:orange">goal</span><br>
        <span style="color:green">package_where_message_is/message_type goal_var_name</span><br>
        <span style="color:red">---</span><br>
        #<span style="color:orange">result</span><br>
        <span style="color:green">package_where_message_is/message_type result_var_name</span><br>
        <span style="color:red">---</span><br>
        #<span style="color:orange">feedback</span><br>
        <span style="color:green">package_where_message_is/message_type feedback_var_name</span>
        <br><br>
        </li>
        <li>如果你不需要message的某部分 (比如你不需要提供反馈) 你可以将该部分留空，但必须始终指定连字符分隔号</li>
      </ul>
  </li>
  <li>修改文件 CMakeLists.txt 和 package.xml 以包含action message编译。查看详细说明 <a href="#custom_compilation">如何准备 CMakeLists.txt 和 package.xml 于自定义action编译</a>
  </li>
</ol>

<br><br>
但一切设置正确，你只需编写:<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #FFFFFF">在 WebShell #1中执行</p><br>
roscd; cd ..<br>
catkin_make<br>
source devel/setup.bash<br>
rosmsg list | grep NAME<br>
</th>
</tr>
</table>
<br><br><br><br><br><br><br><br>


<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #FFFFFF">Output in WebShell #1</p><br>
my_custom_action_msg_pkg/NAMEAction<br>
my_custom_action_msg_pkg/NAMEActionFeedback<br>
my_custom_action_msg_pkg/NAMEActionGoal<br>
my_custom_action_msg_pkg/NAMEActionResult<br>
my_custom_action_msg_pkg/NAMEFeedback<br>
my_custom_action_msg_pkg/NAMEGoal<br>
my_custom_action_msg_pkg/NAMEResult<br>
</th>
</tr>
</table>
<br><br><br><br><br><br><br><br><br><br><br>





</th>
</tr>
</table>

<div id="custom_compilation"></div>

## 如何准备 CMakeLists.txt 和 package.xml 于自定义action编译

您必须在包中编辑两个文件，如Topic和services中所述：
<ul>
<li>CMakeLists.txt</li>
<li>package.xml</li>
</ul>

### 在 package.xml:

<ol>
<li>
添加所有需要编译messages的包。<br>
打个比方，如果你.action文件中的一个变量使用了一个被定义在std_msgs包之外的message，比如 nav_msgs/Odometry，你需要输入它。要这么做，你就必须添加  <b>build_depend</b> the package <b>nav_msgs</b> package 像这样:
<span style="color:green">&lt;build_depend&gt;nav_msgs&lt;/build_depend&gt;</span>
</li>
<li>
另一方面如果你需要一个包来执行你包中的程序，那么你必须将输入这些包如同 <b>run_depend</b> 像这样:<br>
<span style="color:green">&lt;run_depend&gt;nav_msgs&lt;/run_depend&gt;</span>
</li>
</ol>

当编译自定义action messages时，它<span style="color: red">总是</span>需要添加<span style="color: red">actionlib_msgs</span> 如同 build_dependency.<br>
<span style="color:red">&lt;build_depend&gt;actionlib_msgs&lt;/build_depend&gt;</span><br>
这是因为这些是用于生成基本action message的messages，比如：<i>goal, feedback and result</i>。<br>

当你使用python时，它<span style="color: red">总是</span>需要添加<span style="color: red">rospy</span> 如同 run_dependency.<br>
<span style="color:red">&lt;run_depend&gt;rospy&lt;/run_depend&gt;</span><br>
这是因为为了运行所有你的python ROS代码，rospy python模块是需要的。

### 在 CMakeLists.txt

你将必须编辑在CMakeLists.txt中的这四个函数:
<ul>
<li><span style="color:green"><a href="#find_package">find_package()</a></span></li>
<li><span style="color:green"><a href="#add_action_files">add_action_files()</a></span></li>
<li><span style="color:green"><a href="#generate_messages">generate_messages()</a></span></li>
<li><span style="color:green"><a href="#catkin_package">catkin_package()</a></span></li>
</ul>


<span style="color: green" id="find_package">find_package()</span>

从这里的所有包需要编译topic，services 和 actions的messages。 它们只获取它们的路径，而不是真的导入它们被用于编译。其仅用于使CMakeLists.txt的更多函数能够找到这些包。<br>
在package.xml中你需要声明它们如build

<span style="color: green" id="add_action_files">add_action_files()</span>

在这个函数中会将所有这个包的action messages( 在 action 文件夹中 ) 进行编译。<br>
你可以声明它获取在action目录中的所有action: DIRECTORY action<br>
或者只是明确声明action messages: FILES my_custom_action.action<br>
在你的情况中，你只需要根据需求做二者之一。

<span style="color: green" id="generate_messages">generate_messages()</span>

这里是导入action messages编译所需的包.

<span style="color: green" id="catkin_package">catkin_package()</span>

在这里所有的包将需要由某人在你的包中执行一些动作。<br>
这里的所有包必须作为run_depend存在于package.xml

你应该有一个像这样的CMakeLists.txt:

In [2]:
And a package.xml file similar to this:

SyntaxError: invalid syntax (<ipython-input-2-d07fd263c008>, line 1)

<p style="background:green;color:white;">注意</p><br>
不是你没有在任何地方导入 <b>std_msgs</b> 包。而是你可以使用在你自定义.actions中声明的message。这是因为这个包package是roscore文件系统的一部分，因此它嵌入编译协议中，并且不需要声明。

<table style="float:left;">
<tr>
<th>
<p style="background:#EE9023;color:white;">练习 4.12: 创建带有自定义action message以移动飞行器的包</p><br>
<ul>
  <li>在你之前创建的包<a href="#ex-4-13">[Ex 4.13]</a>中，创建一个新的action server给四轴飞行器</li>
  <li>Action server将接受到如同目标的两个词: UP or DOWN</li>
  <li>当action server接收到UP时，它将使无人机向上移动1米</li>
  <li>当action server接受到DOWN时，它将使无人机向下移动1米</li>
  <li>作为反馈，它将每秒发布它正在执行什么动作(上升或下降)</li>
  <li>当结束动作后，结果将不反应任何东西</li>
</ul>

<p style="background:#3B8F10;color:white;">数据</p><br>
<ul>
<li>你需要创建一个新的具有指定值为<i>String</i>的action message。此类型可以从 <i>std_msgs</i> 包导入。</li>
<li>Action  message的结果部分将是空的。</li>
<li>由于我们谈论的是无人机，你可以在三个轴上指定速度。您将需要这样做，以便上下移动机器人。</li>
</ul>
</th>
</tr>
</table>